# 05 Vulnerability and policy scenarios
This notebook covers the vulnerability index, sensitivity, and
policy targeting scenarios. It reads prepared outputs and runs
policy steps only if needed.

In [ ]:
from pathlib import Path
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

ROOT = Path("..").resolve()
CONFIG = ROOT / "config.yaml"
PYTHON = ROOT / ".venv" / "bin" / "python"
AUTO_RUN = False

try:
    from IPython import get_ipython

    _IN_NOTEBOOK = get_ipython() is not None
except Exception:
    _IN_NOTEBOOK = False

if not _IN_NOTEBOOK:
    import matplotlib

    matplotlib.use("Agg")


def run_cmd(args):
    subprocess.run(args, check=True, cwd=ROOT)


def ensure_artifacts(paths, auto_run=True):
    missing = [p for p in paths if not p.exists()]
    if missing and auto_run:
        run_cmd([str(PYTHON), "-m", "run", "policy", "indice", "--config", str(CONFIG)])
        run_cmd([str(PYTHON), "-m", "run", "policy", "sensibilidad", "--config", str(CONFIG)])
        run_cmd([str(PYTHON), "-m", "run", "policy", "escenarios", "--config", str(CONFIG)])
    return missing


indice_path = ROOT / "data" / "processed" / "indice_vulnerabilidad.parquet"
scen_a = ROOT / "outputs" / "policy" / "escenario_A_beneficiarios.csv"
scen_b = ROOT / "outputs" / "policy" / "escenario_B_beneficiarios.csv"
ensure_artifacts([indice_path, scen_a, scen_b], auto_run=AUTO_RUN)

## Vulnerability index
Higher values indicate higher vulnerability based on level and shock exposure.

In [ ]:
idx = pd.read_parquet(indice_path)
year = int(idx["anio"].max())
idx_y = idx[idx["anio"] == year]
idx_y[["ubigeo", "vulnerabilidad"]].head()

In [ ]:
vals = idx_y["vulnerabilidad"].dropna()
if vals.empty:
    print("No vulnerability values available for the latest year.")
else:
    plt.hist(vals, bins=20)
    plt.title("Vulnerability distribution")
    plt.show()

## Sensitivity analysis
This table shows rank stability across alternate weights.

In [ ]:
import pandas as pd
sens_path = ROOT / "outputs" / "tables" / "sensibilidad_indice.csv"
if sens_path.exists():
    pd.read_csv(sens_path)
else:
    print("Sensitivity table not found")

## Scenario outputs
Scenarios apply targeting rules on the latest year.

In [ ]:
scen_a_df = pd.read_csv(scen_a)
scen_b_df = pd.read_csv(scen_b)
scen_a_df.head()

In [ ]:
coverage = pd.DataFrame(
    {
        "escenario": ["A", "B"],
        "cobertura": [scen_a_df["beneficiario"].mean(), scen_b_df["beneficiario"].mean()],
    }
)
coverage

## Map of beneficiaries (Scenario A)

In [ ]:
limites = gpd.read_file(ROOT / "data" / "geo" / "dim_territorio_base.gpkg")
scen_y = scen_a_df[scen_a_df["anio"] == year][["ubigeo", "beneficiario"]]
limites["ubigeo"] = limites["ubigeo"].astype(str).str.zfill(6)
scen_y["ubigeo"] = scen_y["ubigeo"].astype(str).str.zfill(6)
merged = limites.merge(scen_y, on="ubigeo", how="left")
ax = merged.plot(column="beneficiario", legend=True, figsize=(5, 5))
ax.set_axis_off()
plt.title("Scenario A beneficiaries")